In [1]:
import dataclasses
import functools
import logging
import os
import sys
from collections.abc import Callable

import openai

from effectful.handlers.llm import Template
from effectful.handlers.llm.providers import (
    CacheLLMRequestHandler,
    LLMLoggingHandler,
    OpenAIAPIProvider,
    llm_request,
    tool_call,
)
from effectful.handlers.llm.synthesis import ProgramSynthesis
from effectful.ops.semantics import fwd, handler
from effectful.ops.syntax import defop

provider = OpenAIAPIProvider(openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

## Interface

The `robotl.ops.llm` module provides a simplified LLM interface that uses algebraic effects to provide modularity. The module interface consists of:

- A decorator `template` which creates a prompt template from a callable. We should think of the prompt template as an LLM-implemented function with behavior specified by a template string. When a templated function is called, an LLM is invoked to produce the specified behavior. The `__call__` method of a template is a handleable operation.
- An operation `decode` which parses LLM output. `decode(t: type, c: str)` converts an LLM response `c` to the type `t`. It can be handled to provide decoding logic for particular types.
- Interpretations for LLM providers `OpenAIIntp` and callable decoding `ProgramSynthesisIntp`. These interpretations can be composed to handle a variety of template behaviors.

## Prompt Templates

This template function writes (bad) poetry on a given theme. While difficult to implement in Python, an LLM can provide a reasonable implementation.

In [2]:
@Template.define
def limerick(theme: str) -> str:
    """Write a limerick on the theme of {theme}."""
    raise NotImplementedError

If we call the template with a provider interpretation installed, we get reasonable behavior. The LLM is nondeterministic by default, so calling the template twice with the same arguments gives us different results.

Templates are regular callables, so can be converted to operations with `defop` if we want to override the LLM implementation in some cases.

In [3]:
with handler(provider):
    print(limerick("fish"))
    print("-" * 40)
    print(limerick("fish"))

In the sea where the swift fish do play,  
A sardine dreamed of swimming away.  
It fancied a flight,  
With wings to take flight,  
But found fins were quite fine for the bay.
----------------------------------------
There once was a fish named Lou,  
Who swam in a sea of bright blue.  
He'd dance with a eel,  
Turning with zeal,  
And sing to the starfish anew.  


If we want deterministic behavior, we can cache the template call. We can either cache it with the default `@functools.cache` or using `CacheLLMRequestHandler`:

In [4]:
@functools.cache
@Template.define
def haiku(theme: str) -> str:
    """Write a haiku on the theme of {theme}."""
    raise NotImplementedError

@Template.define
def haiku_no_cache(theme: str) -> str:
    """Write a haiku on the theme of {theme}."""
    raise NotImplementedError


print()
with handler(provider):
    print(haiku("fish"))
    print("-" * 40)
    print(haiku("fish"))

print()
cache_handler1 = CacheLLMRequestHandler(silent_failure=False)
with handler(provider), handler(cache_handler1):
    print(haiku_no_cache("fish2"))
    print("-" * 40)
    print(haiku_no_cache("fish2"))

print()
cache_handler2 = CacheLLMRequestHandler()
with handler(provider), handler(cache_handler2):
    print(haiku_no_cache("fish2"))
    print("-" * 40)
    print(haiku_no_cache("fish2"))




Stream whispers softly,  
Silver fins dart through the light—  
Dance of fleeting dreams.
----------------------------------------
Stream whispers softly,  
Silver fins dart through the light—  
Dance of fleeting dreams.

Fish2 glides with grace,  
Digital waves ripple wide,  
Silent currents hum.
----------------------------------------
Fish2 glides with grace,  
Digital waves ripple wide,  
Silent currents hum.

Scales glint in sunlight,  
Silent currents cradle dreams—  
Mysteries beneath.
----------------------------------------
Scales glint in sunlight,  
Silent currents cradle dreams—  
Mysteries beneath.


## Converting LLM Results to Python Objects

Type conversion is handled by `decode`. By default, primitive types are converted. `DecodeError` is raised if a response cannot be converted.

In [5]:
@Template.define
def primes(first_digit: int) -> int:
    """Give a prime number with {first_digit} as the first digit."""
    raise NotImplementedError


with handler(provider):
    assert type(primes(6)) is int

More complex types can be converted by providing handlers for `decode`. `ProgramSynthesisIntp` provides a `decode` handler that parses Python callables.

In [6]:
@Template.define
def count_char(char: str) -> Callable[[str], int]:
    """Write a function which takes a string and counts the occurrances of '{char}'."""
    raise NotImplementedError


with handler(provider), handler(ProgramSynthesis()):
    count_a = count_char("a")
    assert callable(count_a)
    assert count_a("banana") == 3
    assert count_a("cherry") == 0

## Tool Calling

Passing `Operation`s to `Template.define` makes them available for the LLM to call as tools. The description of these operations is inferred from their type annotations and docstrings.

Tool calls are mediated by a helper operation `tool_call`. Handling this operation allows tool use to be tracked or logged.

In [7]:
@defop
def cities() -> list[str]:
    return ["Chicago", "New York", "Barcelona"]


@defop
def weather(city: str) -> str:
    status = {"Chicago": "cold", "New York": "wet", "Barcelona": "sunny"}
    return status.get(city, "unknown")


@Template.define(tools=[cities, weather])
def vacation() -> str:
    """Use the provided tools to suggest a city that has good weather."""
    raise NotImplementedError


def log_tool_call(_, tool, *args, **kwargs):
    result = fwd()
    print(f"Tool call: {tool}(*{args}, **{kwargs}) -> {result}")
    return result


with handler(provider), handler({tool_call: log_tool_call}):
    print(vacation())

Tool call: cities(*(), **{}) -> ['Chicago', 'New York', 'Barcelona']
Tool call: weather(*(), **{'city': 'Chicago'}) -> cold
Tool call: weather(*(), **{'city': 'New York'}) -> wet
Tool call: weather(*(), **{'city': 'Barcelona'}) -> sunny
Tool call: weather(*(), **{'city': 'Chicago'}) -> cold
Tool call: weather(*(), **{'city': 'New York'}) -> wet
Tool call: weather(*(), **{'city': 'Barcelona'}) -> sunny
I'm currently unable to retrieve the weather information for these cities. Could you try again later or provide another way to assist you?


## Structured Output Generation

Constrained generation is used for any type that is convertible to a Pydantic model.

In [8]:
@dataclasses.dataclass
class KnockKnockJoke:
    whos_there: str
    punchline: str


@Template.define
def write_joke(theme: str) -> KnockKnockJoke:
    """Write a knock-knock joke on the theme of {theme}."""
    raise NotImplementedError


@Template.define
def rate_joke(joke: KnockKnockJoke) -> bool:
    """Decide if {joke} is funny or not"""
    raise NotImplementedError


def do_comedy():
    joke = write_joke("lizards")
    print("> You are onstage at a comedy club. You tell the following joke:")
    print(
        f"Knock knock.\nWho's there?\n{joke.whos_there}.\n{joke.whos_there} who?\n{joke.punchline}"
    )
    if rate_joke(joke):
        print("> The crowd laughs politely.")
    else:
        print("> The crowd stares in stony silence.")


with handler(provider):
    do_comedy()

> You are onstage at a comedy club. You tell the following joke:
Knock knock.
Who's there?
Iguana.
Iguana who?
Iguana tell you a secret—I'm not really a door-to-door lizard!
> The crowd laughs politely.


### Logging LLM requests
To intercept messages being called on the lower-level, we can write a handler for `llm_request`:

In [9]:
def log_llm(client, *args, **kwargs):
    result = fwd()
    print("Request fired: ", args, kwargs, result)
    return result


# Avoid cache
try:
    haiku.cache_clear()
except Exception:
    pass

# Put llm_request handler innermost so it has highest precedence during the call
with handler(provider), handler({llm_request: log_llm}):
    _ = haiku("fish2")
    _ = limerick("fish")  # or use haiku("fish-2") to avoid cache

Request fired:  () {'input': [{'type': 'message', 'content': [{'type': 'input_text', 'text': 'Write a haiku on the theme of fish2.'}], 'role': 'user'}], 'model': 'gpt-4o', 'tools': [], 'tool_choice': 'auto'} Response(id='resp_001c91e7128de92400690f38cfb68c8193858f2196a95cc024', created_at=1762605263.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_001c91e7128de92400690f38d06dd88193add06ad5b9aab16d', content=[ResponseOutputText(annotations=[], text="Beneath ripples' dance,  \nSilent swirls of life and grace—  \nDreams in liquid gold.  ", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=Non

### Python logging for LLM requests and tool calls
We can also uses Python logger through `LLMLoggingHandler` to log both low-level LLM requests (`llm_request`) and model-initiated tool use (`tool_call`):


In [10]:
# 1. Create a logger
logger = logging.getLogger("effectful.llm")
logger.setLevel(logging.INFO)
log_handler = logging.StreamHandler(sys.stdout)
log_handler.setFormatter(logging.Formatter("%(levelname)s %(payload)s"))
logger.addHandler(log_handler)
# 2. Pass it to the handler
llm_logger = LLMLoggingHandler(logger=logger)  # can also be LLMLoggingHandler()

# Avoid cache for demonstration
try:
    haiku.cache_clear()
    limerick.cache_clear()
except Exception:
    pass

with handler(provider), handler(llm_logger):
    _ = haiku("fish3")
    _ = limerick("fish4")

INFO {'args': (), 'kwargs': {'input': [{'type': 'message', 'content': [{'type': 'input_text', 'text': 'Write a haiku on the theme of fish3.'}], 'role': 'user'}], 'model': 'gpt-4o', 'tools': [], 'tool_choice': 'auto'}, 'response': Response(id='resp_07f89439c92725ee00690f38d26a048194902c3090bffc4a55', created_at=1762605266.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_07f89439c92725ee00690f38d2ce1c8194986668932eb6e573', content=[ResponseOutputText(annotations=[], text='Silent waters glide,  \nSilver whispers beneath waves—  \nFish dance in moonlight.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Rea